In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import erfc
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks

In [ ]:
plt.style.use('darkside')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
# Sample rate and desired cutoff frequencies (in Hz).
fs = 250000000.0
lowcut = 10.0
highcut = 8000000.0

In [ ]:
def gauss(x,a,b,c):
    return a*np.exp(-(x-b)**2/(2*c**2))

def line(x,a,b):
    return a*x+b

In [ ]:
voltages = ['20220923_60V_laser_3725thres','20220923_61V_laser_3722thres','20220923_62V_laser_3722thres','20220923_63V_laser_3720thres','20220923_64V_laser_3718thres']

In [ ]:
for ch in [0,1,2,3]:
    for volt in voltages: 
        print(ch, volt)
        
        fname = "/home/aj9512/jadwin-365/data/sipm/{}/wave{}.dat".format(volt,ch)
        dt = np.dtype('<H')
        file = open(fname, 'rb')
        traces = np.fromfile(file, dtype=dt, count=-1)
        file.close()

        traces = traces.reshape((-1,2048)).astype(float)
        tt = np.arange(0,0.004*2048,0.004)

        amp = [] 
        integral = []

        for i,wvf in enumerate(traces):
            if i % 10000 == 0:
                print(i)
            traces[i] = traces[i] - np.mean(traces[i][:100])
            traces[i] = butter_bandpass_filter(traces[i], lowcut, highcut, fs, order=3)
            amp.append(np.min(traces[i]))
            integral.append(np.sum(traces[i]))
        
        amp = np.array(amp)
        integral = np.array(integral)

        h,hx = np.histogram(np.abs(amp), bins=np.arange(0,100,.1))
        pp,pdict = find_peaks(h, prominence=100, width=10, distance=5)
        fig, ax = plt.subplots(figsize=(9,3), ncols=2,nrows=1)
        ww = 10
        fit_p = []
        fit_c = []

        for x in pp: 
            popt,pcov = curve_fit(gauss, hx[:-1][x-ww:x+ww], h[x-ww:x+ww], p0=[h[x], hx[:-1][x], 2], maxfev=10000)
            ax[0].plot(hx[:-1][x-ww:x+ww], gauss(hx[:-1][x-ww:x+ww], *popt), color=colors[1], lw=1, zorder=10)
            fit_p.append(popt)
            fit_c.append(pcov)

        ax[0].step(hx[:-1],h, where='post')
        ax[0].scatter(hx[pp], h[pp], color='r', s=3, zorder=10)
        ax[0].set_xlabel('Amplitude [mV]')
        ax[0].set_ylabel('Counts')
        ax[0].set_yscale('log')
        ax[0].set_xlim(0,30)
        ax[0].set_ylim(1e0,1e4)

        popt,pcov = curve_fit(line, np.arange(1,len(fit_p)+1,1), np.array(fit_p)[:,1])
        perr = np.sqrt(np.diag(pcov))
        print(perr)
        xfit = np.linspace(0,20,100)

        gain = np.mean(np.diff(np.array(fit_p)[:,1]))
        print(gain)
        print(pp)

        ax[1].grid()
        ax[1].plot(xfit, line(xfit, *popt), color=colors[1], ls='--', label=r'$m={:.3f}\pm {:.3f}$/p.e.'.format(popt[0], perr[0]))
        ax[1].scatter(np.arange(1,len(fit_p)+1,1), np.array(fit_p)[:,1])
        ax[1].set_xlabel('Number of p.e.')
        ax[1].set_ylabel('Amplitude [a.u.]')
        ax[1].set_xlim(0,10)
        ax[1].set_ylim(0,20)
        ax[1].legend(loc='lower right')

        plt.show()

In [ ]:
ch = 0
# fname = "/home/aj9512/jadwin-365/data/sipm/20220923_60V_laser_3725thres/wave{}.dat".format(ch)
# fname = "/home/aj9512/jadwin-365/data/sipm/20220923_61V_laser_3722thres/wave{}.dat".format(ch)
# fname = "/home/aj9512/jadwin-365/data/sipm/20220923_62V_laser_3722thres/wave{}.dat".format(ch)
# fname = "/home/aj9512/jadwin-365/data/sipm/20220923_63V_laser_3720thres/wave{}.dat".format(ch)
# fname = "/home/aj9512/jadwin-365/data/sipm/20220923_64V_laser_3718thres/wave{}.dat".format(ch)
fname = "/home/aj9512/jadwin-365/data/sipm/20220923_60V_scint_3710thres/wave{}.dat".format(ch)

In [ ]:
dt = np.dtype('<H')
file = open(fname, 'rb')
traces = np.fromfile(file, dtype=dt, count=-1)
file.close()

In [ ]:
traces = traces.reshape((-1,2048)).astype(float)
tt = np.arange(0,0.004*2048,0.004)

In [ ]:
y = butter_bandpass_filter(traces[1000], lowcut, highcut, fs, order=3)

plt.figure()
plt.plot(tt, traces[1000]-np.mean(traces[1000][:100]), label='original')
plt.plot(tt, y-np.mean(y[100:200]), label='filtered')
plt.xlabel('Time [$\mu$s]')
plt.ylabel('Amplitude [mV]')
plt.xlim(2,4)
plt.ylim(-50, 10)
plt.legend(loc='upper right')
plt.show()

In [ ]:
amp = [] 
integral = []

for i,wvf in enumerate(traces):
    if i % 10000 == 0:
        print(i)
    traces[i] = traces[i] - np.mean(traces[i][:100])
    traces[i] = butter_bandpass_filter(traces[i], lowcut, highcut, fs, order=3)
    amp.append(np.min(traces[i]))
    integral.append(np.sum(traces[i]))

In [ ]:
amp = np.array(amp)
integral = np.array(integral)

In [ ]:
plt.figure()

for i,wvf in enumerate(traces[::1000]):
    plt.plot(tt, wvf, lw=1, color='k', alpha=0.1)
# plt.plot(tt, np.mean(traces, axis=0), color='r')
# plt.vlines(x=0.4*np.max(tt), ymin=-100, ymax=20, linestyles='--', lw=1)

plt.xlabel('Time [$\mu$s]')
plt.ylabel('Amplitude [mV]')
plt.xlim(2,8)
plt.ylim(-500, 10)
plt.show()

In [ ]:

h,hx = np.histogram(np.abs(amp), bins=np.arange(0,1000,.1))

pp,pdict = find_peaks(h, prominence=100, width=10, distance=5)

fig, ax = plt.subplots(figsize=(9,3), ncols=2,nrows=1)
ww = 10
fit_p = []
fit_c = []

# for x in pp: 
#     popt,pcov = curve_fit(gauss, hx[:-1][x-ww:x+ww], h[x-ww:x+ww], p0=[h[x], hx[:-1][x], 2], maxfev=10000)
#     ax[0].plot(hx[:-1][x-ww:x+ww], gauss(hx[:-1][x-ww:x+ww], *popt), color=colors[1], lw=1, zorder=10)
#     fit_p.append(popt)
#     fit_c.append(pcov)

ax[0].step(hx[:-1],h, where='post')
ax[0].scatter(hx[pp], h[pp], color='r', s=3, zorder=10)
ax[0].set_xlabel('Amplitude [mV]')
ax[0].set_ylabel('Counts')
ax[0].set_yscale('log')
ax[0].set_xlim(0,50)
ax[0].set_ylim(1e0,1e4)

# popt,pcov = curve_fit(line, np.arange(1,len(fit_p)+1,1), np.array(fit_p)[:,1])
# perr = np.sqrt(np.diag(pcov))
# print(perr)
# xfit = np.linspace(0,20,100)

# gain = np.mean(np.diff(np.array(fit_p)[:,1]))
# print(gain)
# print(pp)

# ax[1].grid()
# ax[1].plot(xfit, line(xfit, *popt), color=colors[1], ls='--', label=r'$m={:.3f}\pm {:.3f}$/p.e.'.format(popt[0], perr[0]))
# ax[1].scatter(np.arange(1,len(fit_p)+1,1), np.array(fit_p)[:,1])
# ax[1].set_xlabel('Number of p.e.')
# ax[1].set_ylabel('Amplitude [a.u.]')
# ax[1].set_xlim(0,10)
# ax[1].set_ylim(0,20)
# ax[1].legend(loc='lower right')

plt.show()

In [ ]:
plt.figure()
norm_all = []
for i,wvf in enumerate(traces[::100]):
    wvf_norm = -1*wvf/np.min(wvf)
    norm_all.append(wvf_norm)
    plt.plot(tt, wvf_norm, lw=1, color='k', alpha=0.01)

mean_norm = np.mean(norm_all, axis=0)
plt.plot(tt, mean_norm, color='r')

plt.xlabel('Time [$\mu$s]')
plt.ylabel('Amplitude [mV]')
plt.xlim(0,np.max(tt))
plt.xlim(3,5)
plt.ylim(-2,2)
plt.show()

In [ ]:
def func(x,a,b,c,d,e):
    return a*np.exp(-x/b)+c*np.exp(-x/d)+e
def func2(x,a,b,c):
    return a*np.exp(-x/b)+c

def func3(x,base,V0,sigma,tau,mu,tau2):
    return base + V0/2.0 * erfc(1.0/np.sqrt(2.0) * (sigma/tau - (x-mu)/sigma)) * ( np.exp(0.5 * (sigma/tau)**2 - (x-mu)/tau) + np.exp(0.5 * (sigma/tau2)**2 - (x-mu)/tau2))

def func4(x,base,V0,sigma,tau,mu):
    return base + V0/2.0 * erfc(1.0/np.sqrt(2.0) * (sigma/tau - (x-mu)/sigma)) * ( np.exp(0.5 * (sigma/tau)**2 - (x-mu)/tau))

popt2,pcov2 = curve_fit(func3, tt, -1*mean_norm, maxfev=10000, p0=[0, 1, 0.1, 0.03, 3.2, 1.], bounds=([-1,0,0,0,3,1],[1,2,2,1,3.5,2]))
xfit = np.linspace(0,10,100000)
print(popt2)

plt.figure()
plt.plot(tt, -1*mean_norm, color='k')
plt.plot(xfit, func3(xfit, *popt2), color='r', ls='-', label=r"$\tau_1={:.3f}\,\mu$s, $\tau_2={:.3f}\,\mu$s".format(popt2[3], popt2[-1]), lw=1)
plt.plot(xfit, func4(xfit,  *popt2[:-1]), color='b', ls='--', label="Singlet", lw=1)
plt.plot(xfit, func4(xfit,  popt2[0], popt2[1], popt2[2], popt2[-1],popt2[4]), color='g', ls='--', label="Triplet", lw=1)

plt.xlabel('Time [$\mu$s]')
plt.ylabel('Amplitude [mV]')
plt.xlim(0,np.max(tt))
plt.xlim(2,8)
plt.ylim(-1,1)
plt.legend(loc='upper right')
plt.show()